# For a three star system sim each star needs:
mass - scalar value 
posistion - as a 3D vector (x, y, z) representing its location
velocity - a 3D vector (vx, vy, vz) to represent current speed and direction

# For calculating gravitational forces:
find the force magnitude of each pair of stars
F = G(m1*m2)/r^2 
each star experiences a net force that is the vector sum of the gravitational forces of the other two

# Updating motion (numarical integration)
approximate the motion over small time steps delta (t).
Acceleration - for each star a = Fnet / m
Velocity update - vnew = vcurrent + a * delta(t)
Position update - rnew = rcurrent + vnew * delta(t)
This is repeated for each step Google recommends Euler's method (simple but less accurate) or more advanced methods like the Runge-Kutta methods (more accurate but complex). Research both see what is your level(to myself).

# Simulation loop
Within the loop:
.Calculate all pairwise gravitational forces
.Calculate the net forces on each star
.Update the acceleration, velocity, and position of each star
.Visualize stars position at each step

# Visualization 
Google says: Libraries like Matplotlib or VPython can be used to visualize the stars' trajectories in 2D or 3D, making the simulation results understandable.

Proposition of three star system is polaris star system
(mass in solar masses)
m1 = 5.4 
m2 = 1.39
m3 = 1.26
Or Alpha Centauri (personal choice)
m1 = 1.1
m2 = 0.9 
m3 = 0.15

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
G = 6.67430e-11 # m^3 kg^-1 s^-2/ This is the gravitational constant

